In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
import malaya_speech.train.model.vggvox_v1 as vggvox_v1
import malaya_speech.train.model.vggvox_v2 as vggvox_v2
import malaya_speech.train.model.deep_speaker as deep_speaker
import numpy as np

In [4]:
input_restnet_v1 = tf.placeholder(tf.float32, (None, 512, None, 1))
resnet_v1 = vggvox_v1.Model(
    input_restnet_v1, num_class = 7, mode = 'train'
)
resnet_v1.logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


<tf.Tensor 'dense/BiasAdd:0' shape=(?, 7) dtype=float32>

In [5]:
input_resnet_v2 = tf.placeholder(tf.float32, (None, 257, None, 1))
resnet_v2 = vggvox_v2.Model(
    input_resnet_v2, num_class = 7, mode = 'train'
)
resnet_v2.logits

<tf.Tensor 'prediction/Softmax:0' shape=(?, 7) dtype=float32>

In [6]:
input_deep_speaker = tf.placeholder(tf.float32, (None, None, 64, 1))
model = deep_speaker.Model(
    input_deep_speaker, num_class = 7, mode = 'train'
)
model.logits

<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 7) dtype=float32>

In [8]:
y, sr = malaya_speech.load('../speech/example-speaker/mas-aisyah.wav')
y = malaya_speech.utils.astype.int_to_float(y)
y, sr

(array([ 0.00781521, -0.00224018, -0.00719304, ...,  0.05068768,
         0.05327342,  0.05282006]),
 16000)

In [9]:
vggvox_v1_feature = malaya_speech.utils.featurization.vggvox_v1(y)
vggvox_v2_feature = malaya_speech.utils.featurization.vggvox_v2(y)
deep_speaker_feature = malaya_speech.utils.featurization.read_mfcc(y, voice_only = False)

In [10]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
%%time

sess.run(resnet_v1.logits, feed_dict = {input_restnet_v1:[np.expand_dims(vggvox_v1_feature, -1)]})

CPU times: user 321 ms, sys: 36.7 ms, total: 358 ms
Wall time: 171 ms


array([[ 0.02058806,  0.05184993, -0.01417861, -0.00809719,  0.00103807,
         0.03611998, -0.07480001]], dtype=float32)

In [12]:
%%time

sess.run(resnet_v2.logits, feed_dict = {input_resnet_v2:[np.expand_dims(vggvox_v2_feature, -1)]})

CPU times: user 993 ms, sys: 154 ms, total: 1.15 s
Wall time: 392 ms


array([[0.13852549, 0.14883365, 0.14696944, 0.14419368, 0.1432571 ,
        0.13477945, 0.14344126]], dtype=float32)

In [13]:
%%time

sess.run(model.logits, feed_dict = {input_deep_speaker:[np.expand_dims(deep_speaker_feature, -1)]})

CPU times: user 469 ms, sys: 28.8 ms, total: 498 ms
Wall time: 237 ms


array([[-0.13765474,  0.55725396, -0.84844697, -0.8568354 , -0.28648818,
         0.53688765,  0.23803732]], dtype=float32)